In [1]:
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 1905/448 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = 3 * TR_C * TR_R
TR_R = tau / (3*TR_C)




# Logic Gates
g_inv, p_inv = 1, 321/127

g_nand2, g_nand3, g_nand6 = 4/3 , 5/3, 8/3
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 5/3, 7/3
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

C_out = (WL_C + cell_C)
C_in = 5 * 1e-15 * 32
H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = ['value'], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,value
NAND6-INV,3.678072
NAND3-NOR2,3.707519


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,value
NAND6-INV Stage2,1.840915e-10
NAND3-NOR2 Stage2,1.648407e-10


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,value
NAND6-INV Stage4,1.575829e-10
NAND3-NOR2 Stage4,1.367116e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND6-INV'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,value
NAND6-INV Stage6,1.779067e-10
NAND3-NOR2 Stage6,1.568186e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,value
NAND6-INV Stage8,2.043944e-10
NAND3-NOR2 Stage8,1.828988e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,value
NAND6-INV Stage2,1.840915e-10
NAND3-NOR2 Stage2,1.648407e-10
NAND6-INV Stage4,1.575829e-10
NAND3-NOR2 Stage4,1.367116e-10
NAND6-INV Stage6,1.779067e-10
NAND3-NOR2 Stage6,1.568186e-10
NAND6-INV Stage8,2.043944e-10
NAND3-NOR2 Stage8,1.828988e-10


In [9]:
# NMOS width of repeater drived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/(3*metal_r*TR_C) ) ** (1/2)

segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (3*TR_C*W) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 15 *1e-12 #measured FO1 delay
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.22002641e-09, 2.80483988e-09, 1.64841136e-09, 1.18645462e-09,
       1.18501211e-09, 1.62311478e-09, 2.67984103e-09, 4.81900644e-09])

In [10]:
delay_total = delay_decoder.copy()
for i in range(7):
  delay_total = pd.concat([delay_total, delay_decoder], axis=1)
delay_total = delay_total + delay_WL
delay_total.columns = [1,2,4,8,16,32,64,128]
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.404118e-09,2.988931e-09,1.832503e-09,1.370546e-09,1.369104e-09,1.807206e-09,2.863933e-09,5.003098e-09
NAND3-NOR2 Stage2,5.384867e-09,2.969681e-09,1.813252e-09,1.351295e-09,1.349853e-09,1.787955e-09,2.844682e-09,4.983847e-09
NAND6-INV Stage4,5.377609e-09,2.962423e-09,1.805994e-09,1.344038e-09,1.342595e-09,1.780698e-09,2.837424e-09,4.976589e-09
NAND3-NOR2 Stage4,5.356738e-09,2.941551e-09,1.785123e-09,1.323166e-09,1.321724e-09,1.759826e-09,2.816553e-09,4.955718e-09
NAND6-INV Stage6,5.397933e-09,2.982747e-09,1.826318e-09,1.364361e-09,1.362919e-09,1.801022e-09,2.857748e-09,4.996913e-09
NAND3-NOR2 Stage6,5.376845e-09,2.961659e-09,1.805230e-09,1.343273e-09,1.341831e-09,1.779933e-09,2.836660e-09,4.975825e-09
NAND6-INV Stage8,5.424421e-09,3.009234e-09,1.852806e-09,1.390849e-09,1.389406e-09,1.827509e-09,2.884235e-09,5.023401e-09
NAND3-NOR2 Stage8,5.402925e-09,2.987739e-09,1.831310e-09,1.369353e-09,1.367911e-09,1.806014e-09,2.862740e-09,5.001905e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.390682e-09,2.975496e-09,1.819067e-09,1.357110e-09,1.355668e-09,1.793770e-09,2.850497e-09,4.989662e-09
std,2.094097e-11,2.094097e-11,2.094097e-11,2.094097e-11,2.094097e-11,2.094097e-11,2.094097e-11,2.094097e-11
min,5.356738e-09,2.941551e-09,1.785123e-09,1.323166e-09,1.321724e-09,1.759826e-09,2.816553e-09,4.955718e-09
25%,5.377418e-09,2.962232e-09,1.805803e-09,1.343846e-09,1.342404e-09,1.780507e-09,2.837233e-09,4.976398e-09
50%,5.391400e-09,2.976214e-09,1.819785e-09,1.357828e-09,1.356386e-09,1.794488e-09,2.851215e-09,4.990380e-09
75%,5.403223e-09,2.988037e-09,1.831608e-09,1.369652e-09,1.368209e-09,1.806312e-09,2.863038e-09,5.002203e-09
max,5.424421e-09,3.009234e-09,1.852806e-09,1.390849e-09,1.389406e-09,1.827509e-09,2.884235e-09,5.023401e-09


In [12]:
H

1.92

In [13]:
f_4

,value
NAND6-INV,3.577709
NAND3-NOR2,3.614408


In [15]:
W

array([0.65074991, 0.92029934, 1.30149981, 1.84059869, 2.60299963,
       3.68119738, 5.20599926, 7.36239476])

In [16]:
2*2.60299963

5.20599926